MIS 285N Cognitive Computing<br>
Final Project<br>
Jerry Che - Jose Guerrero - Riley Moynihan - Noah Placke - Sarah Teng - Palmer Wenzel

# Ingredients Generation Model (Wasserstein)

Following techniques from:
- https://towardsdatascience.com/generative-adversarial-networks-in-python-73d3972823d3
- https://www.maskaravivek.com/post/gan-synthetic-data-generation/
- (for Wasserstein modifications) https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/

#### Read data from CSV.

In [1]:
import pandas as pd
# pd.options.display.max_columns = 500


df = pd.read_csv('../data/kaggle/processed/recipes_processed.csv')#.sample(frac=0.1, random_state=42)

df.head(3)

,name,steps,crabmeat,creamcheese,greenonions,garlicsalt,refrigeratedcrescentdinnerrolls,eggyolk,water,sesameseeds,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,crab filled crescent snacks,"heat over to 375 degrees, spray large cookie s...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,curried bean salad,"drain & rinse beans, stir all ingredients toge...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,delicious steak with onion marinade,heat the oil in a heavy-based pan and cook the...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Drop unnecessary columns.

In [2]:
df = df.drop(['name', 'steps'], axis=1)

df.head()

,crabmeat,creamcheese,greenonions,garlicsalt,refrigeratedcrescentdinnerrolls,eggyolk,water,sesameseeds,sweetandsoursauce,garbanzobeans,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Train and test split.

In [3]:
from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(df, test_size=0.125, random_state=0)

print(X_train.shape)
print(X_test.shape)

(20268, 7684)
(2896, 7684)


## Model Definition

#### Build functions and model definitions.

Custom loss for Wasserstein implementation.

In [42]:
from keras import backend


def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)

Weight clipping for Wasserstein.

In [43]:
from keras.constraints import Constraint


# Clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value

	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)

	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [46]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, Input
from tensorflow.keras.optimizers import RMSprop


# Model configs
noise_dim = 1000
dim = 256
data_dim = df.shape[1]


def build_generator():
    generator = Sequential()
    
    generator.add(Dense(dim, input_dim=noise_dim))
    generator.add(Dense(dim, activation='relu'))
    generator.add(Dense(dim * 2, activation='relu'))
    generator.add(Dense(dim * 4, activation='relu'))
    generator.add(Dense(data_dim, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer='adam')
    
    return generator


def build_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(dim * 4, input_dim=data_dim))
    discriminator.add(Dropout(0.1))
    discriminator.add(Dense(dim * 2, activation='relu', kernel_constraint=ClipConstraint(0.01)))
    discriminator.add(Dropout(0.1))
    discriminator.add(Dense(dim, activation='relu', kernel_constraint=ClipConstraint(0.01)))  # weight clip constraint for Wassersteign
    discriminator.add(Dense(1, activation='linear'))  # was sigmoid for normal DCGAN, linear for Wasserstein
    
    discriminator.compile(loss=wasserstein_loss, optimizer='adam')  # loss was binary_crossentropy, now custom for Wasserstein
    
    return discriminator


def build_gan(generator, discriminator):
    # Only train generator in combined model
    discriminator.trainable=False
    
    gan_input = Input(shape=(noise_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    
    # Create the GAN model
    gan = Model(inputs=gan_input, outputs=gan_output)
    
    # Use RMSProp for optimizer for Wasserstein
    opt = RMSprop(lr=0.0005)
                      
    gan.compile(loss='binary_crossentropy', optimizer=opt)  # was Adam for DCGAN, now RMSProp for Wasserstein
    
    return gan


generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

gan.summary()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 1000)]            0         
_________________________________________________________________
sequential_40 (Sequential)   (None, 7684)              8855044   
_________________________________________________________________
sequential_41 (Sequential)   (None, 1)                 8525825   
Total params: 17,380,869
Trainable params: 8,855,044
Non-trainable params: 8,525,825
_________________________________________________________________


#### Create function to display generator output.

In [53]:
import numpy as np


def display_recipe(epoch, generator, examples=1):
    # Create noise
    # noise = np.random.normal(0, 1, size=[examples, noise_dim])
    # noise = np.random.laplace(0, 1, size=[examples, noise_dim])
    noise = random_noise(examples)
    
    # Generate recipes
    generated_recipes = generator.predict(noise)
    
    # Get used ingredients
    ingredients = []
    for i in range(generated_recipes.shape[0]):
        for j in range(len(generated_recipes[i])):
            if generated_recipes[i][j] >= 0.9:
                
                ingredients.append(df.columns[j])
    
    # Display
    print("*** Generated Recipe ***")
    print(f"# of ingredients: {len(ingredients)}")
    print(f"Ingredients: {ingredients[:]}")
    print(generated_recipes[i])

In [51]:
import random


def random_noise(num):
    return np.random.normal(0, 1, size=[num, noise_dim])
    # return np.random.laplace(0, 1, size=[num, noise_dim])
    noise = np.random.laplace(0, 1, size=[num, noise_dim])
    for i in range(len(noise)):
        for j in range(len(noise[i])):
            if random.random() < 0.9:
                noise[i][j] = 0.0
    
    return noise

#### Training logic.

In [ ]:
from tqdm import tqdm


def training(X_train, X_test, epochs=1, batch_size=32, sample_interval=10, n_critic=5):
    # Get batch count
    batch_count = X_train.shape[0] / batch_size
    
    # Build GAN
    generator = build_generator()
    discriminator = build_discriminator()
    gan = build_gan(generator, discriminator)
    
    # Training step
    for e in range(1, epochs + 1):
        # for _ in tqdm(range(batch_size)):
            
        # Train 'critic' more than generator
        for _ in range(n_critic):
            
            # Random noise as an input to initialize the generator
            noise = random_noise(batch_size)
            # replace with Laplace?
            # replace high% of noise with 0

            # Use the GAN to generate "fake" recipes
            generated_recipes = generator.predict(noise)

            # Get a sample of real recipes from data
            # real_recipes = X_train.loc[np.random.randint(low=0, high=X_train.shape[0], size=batch_size)]
            real_recipes = X_train.sample(batch_size)

            # Mix the real and fake data
            X = np.concatenate([real_recipes, generated_recipes])

            # Create labels for real and fake data
            y_dis = np.zeros(2 * batch_size)  # fake
            y_dis[:batch_size] = 1.0          # real
            y_dis[batch_size:] = -1.0  # modification for Wasserstein

            # Train the discriminator while generator is fixed
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

        # Fix the images generated by the generator as real
        noise = random_noise(batch_size)
        y_gen = np.ones(batch_size)

        # Train the generator (to have the discriminator label samples as valid)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)

        # Output loss
        print(f"E{e} [D Loss: {d_loss:.4f}] [G loss: {g_loss:.4f}]")
            
        # Display created recipes at a given epoch interval
        if e % sample_interval == 0:
            # Display recipe
            display_recipe(e, generator)
    
    return generator, discriminator, gan


generator, discriminator, gan = training(X_train, X_test, epochs=5000, batch_size=8, n_critic=5)

E1 [D Loss: -5.3392] [G loss: 0.0000]
E2 [D Loss: -63.8278] [G loss: 0.0000]
E3 [D Loss: -283.9616] [G loss: 0.0000]
E4 [D Loss: -940.1898] [G loss: 0.0000]
E5 [D Loss: -1802.8828] [G loss: 0.0000]
E6 [D Loss: -2738.5811] [G loss: 0.0000]
E7 [D Loss: -3715.8013] [G loss: 0.0000]
E8 [D Loss: -4459.5244] [G loss: 0.0000]
E9 [D Loss: -5570.0811] [G loss: 0.0000]
E10 [D Loss: -7385.0762] [G loss: 0.0000]
*** Generated Recipe ***
# of ingredients: 0
Ingredients: []
[-0.25018764  0.04995698  0.18479747 ...  0.02406952  0.13609901
 -0.01225805]
E11 [D Loss: -7952.4385] [G loss: 0.0000]
E12 [D Loss: -9703.9941] [G loss: 0.0000]
E13 [D Loss: -10907.8027] [G loss: 0.0000]
E14 [D Loss: -12240.6680] [G loss: 0.0000]
E15 [D Loss: -15046.4805] [G loss: 0.0000]
E16 [D Loss: -15494.7363] [G loss: 0.0000]
E17 [D Loss: -17651.7441] [G loss: 0.0000]
E18 [D Loss: -20546.3086] [G loss: 0.0000]
E19 [D Loss: -21197.7734] [G loss: 0.0000]
E20 [D Loss: -24604.7383] [G loss: 0.0000]
*** Generated Recipe ***
# o